In [1]:
%pip install qgear-lightning -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 MB 5.6 MB/s eta 0:00:00


In [6]:
import os
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qgear_lightning.core import qiskit_to_gateList, counts_cudaq_to_qiskit, circ_kernel
from qgear_lightning.io import write4_data_hdf5, read4_data_hdf5
import cudaq
observable_1 = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])

In [7]:
def expect_cudaq(gateD, hamiltonian, backend="qpp-cpu"):
    """
    Run CUDA-Q simulation for all circuits in gateD and compute expectation values.

    Parameters
    ----------
    gateD : dict
        Dictionary containing circuit definitions ('circ_type', 'gate_type', 'gate_param').
    hamiltonian : cudaq.SpinOperator
        Hamiltonian for expectation value calculation.
    verb : int
        Verbosity level. >1 prints all circuits, 1 prints only first small one.
    backend : str
        CUDA-Q backend target (e.g., 'qpp-cpu', 'nvidia').

    Returns
    -------
    exp_vals : list of float
        Expectation values for each circuit.
    """
    cudaq.set_target(backend)
    nc = len(gateD['circ_type'])
    exp_vals = [0.0] * nc

    for i in range(nc):
        num_qubit, num_gate = map(int, gateD['circ_type'][i])
        gate_type = list(map(int, gateD['gate_type'][i].flatten()))
        gate_param = list(map(float, gateD['gate_param'][i].flatten()))
        assert num_gate <= len(gate_param)

        # Compute expectation value
        result = cudaq.observe(circ_kernel, hamiltonian, num_qubit, num_gate, gate_type, gate_param)
        exp_vals[i] = result.expectation()

    return exp_vals

In [8]:
def cost_func_vqe_cudaq(params, ansatz_template, hamiltonian, estimator):
    # Bind new parameters into the ansatz
    qc_bound = ansatz_template.assign_parameters(params)

    # Convert to CUDA-Q gate list
    gateD, md = qiskit_to_gateList([qc_bound])

    # Evaluate expectation value
    cost = expect_cudaq(gateD, hamiltonian)
    return cost

In [11]:
from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit

reference_circuit = QuantumCircuit(2)
reference_circuit.x(0)

variational_form = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)
raw_ansatz = reference_circuit.compose(variational_form)


qc = raw_ansatz.decompose()

/tmp/ipython-input-3031976925.py:7: DeprecationWarning: The class ``qiskit.circuit.library.n_local.two_local.TwoLocal`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.n_local instead.
  variational_form = TwoLocal(


In [13]:
# SciPy minimizer routine
from scipy.optimize import minimize
from cudaq import spin
from qiskit.primitives import StatevectorEstimator as Estimator

estimator = Estimator()
x0 = np.ones(raw_ansatz.num_parameters)

# The CUDA Quantum version of the Qiskit SparsePauliOp
observable_1 = 2.0 - 2.0 * spin.x(0) * spin.x(1) + 3.0 * spin.y(0) * spin.y(1) - 3.0 * spin.z(0) * spin.z(1)
start_time = time.time()
result = minimize(
    cost_func_vqe_cudaq,
    x0,
    args=(qc, observable_1, estimator),
    method="COBYLA",
    options={"maxiter": 1000, "disp": True},
)

qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gateList: nGate 10
qiskit_to_gate